In [9]:
import numpy as np
import pandas as pd
from scipy import optimize
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng

import simpy

### Possible improvements

Now that we have a rough first model working, let's think about some possible improvements. There are many as we've taken numerous shortcuts to get this model working.

* Specifying the global sim inputs through configuration files
* Getting rid of hard coded processing time distributions
* Having ability to choose between pure random walk-in arrivals and scheduled arrivals
* Multiple replications and/or steady state analysis
* Detailed logging
* Statistical summaries based on timestamp data [roughly done]
* CLI for running
* Animation


## Model 3: The vaccine clinic model - version 0.01

Here's the basic vaccination process we'll model.

    Arrival --> Temperature check --> Registration --> Vaccination --> Schedule dose 2 (if needed) --> Wait 15 min --> Exit

### Creating the simulation environment



In [10]:
class VaccineClinic(object):
    def __init__(self, env, num_greeters, num_reg_staff, num_vaccinators, num_schedulers, rg):
        # Simulation environment
        self.env = env
        
        # Create list to hold timestamps dictionaries (one per patient)
        self.timestamps_list = []
        # Create lists to hold occupancy tuples (time, occ)
        self.postvac_occupancy_list = [(0.0, 0.0)]
        self.vac_occupancy_list = [(0.0, 0.0)]
        
        # Create resources
        self.greeter = simpy.Resource(env, num_greeters)
        self.reg_staff = simpy.Resource(env, num_reg_staff)
        self.vaccinator = simpy.Resource(env, num_vaccinators)
        self.scheduler = simpy.Resource(env, num_schedulers)

    # Create process methods - hard coding processing time distributions for now
    # The patient argument is just a unique integer number
    def temperature_check(self, patient, rg):
        yield self.env.timeout(rg.normal(0.25, 0.05))

    def registration(self, patient, rg):
        yield self.env.timeout(rg.exponential(1.0))

    def vaccinate(self, patient, rg):
        yield self.env.timeout(rg.normal(4.0, 0.5))
        
    def schedule_dose_2(self, patient, rg):
        yield self.env.timeout(rg.normal(1.0, 0.25))
    
    # We assume all patients wait at least 15 minutes post-vaccination
    # Some will choose to wait longer. This is the time beyond 15 minutes
    # that patients wait.
    def wait_gt_15(self, patient, rg):
        yield self.env.timeout(rg.exponential(0.5))

Now create general function to define the sequence of steps traversed by patients. We'll also capture a bunch of timestamps to make it easy to compute various system performance measures such as patient waiting times, queue sizes and resource utilization.

In [11]:
def get_vaccinated(env, patient, clinic, pct_first_dose, rg):
    # Patient arrives to clinic - note the arrival time
    arrival_ts = env.now

    # Request a greeter for temperature check
    # By using request() in a context manager, we'll automatically release the resource when done
    with clinic.greeter.request() as request:
        yield request
        # Now that we have a greeter, check temperature. Note time.
        got_greeter_ts = env.now
        yield env.process(clinic.temperature_check(patient, rg))
        release_greeter_ts = env.now

    # Request reg staff to get registered
    with clinic.reg_staff.request() as request:
        yield request
        got_reg_ts = env.now
        yield env.process(clinic.registration(patient, rg))
        release_reg_ts = env.now
        
    # Request clinical staff to get vaccinated
    with clinic.vaccinator.request() as request:
        yield request
        got_vaccinator_ts = env.now
        # Update vac occupancy - increment by 1
        prev_occ = clinic.vac_occupancy_list[-1][1]
        new_occ = (env.now, prev_occ + 1)
        clinic.vac_occupancy_list.append(new_occ)
        yield env.process(clinic.vaccinate(patient, rg))
        release_vaccinator_ts = env.now
        # Update vac occupancy - decrement by 1 - more compact code
        clinic.vac_occupancy_list.append((env.now, clinic.vac_occupancy_list[-1][1] - 1))
        
        # Update postvac occupancy - increment by 1
        clinic.postvac_occupancy_list.append((env.now, clinic.postvac_occupancy_list[-1][1] + 1))

    # Request scheduler to schedule second dose if needed
    if rg.random() < pct_first_dose:
        with clinic.scheduler.request() as request:
            yield request
            got_scheduler_ts = env.now
            yield env.process(clinic.schedule_dose_2(patient, rg))
            release_scheduler_ts = env.now
    else:
        got_scheduler_ts = pd.NA
        release_scheduler_ts = pd.NA
        
    # Wait at least 15 minutes from time we finished getting vaccinated 
    post_vac_time = env.now - release_vaccinator_ts
    if post_vac_time < 15:
        # Wait until 15 total minutes post vac
        yield env.timeout(15 - post_vac_time)
        # Wait random amount beyond 15 minutes
        yield env.process(clinic.wait_gt_15(patient, rg))
        exit_system_ts = env.now
        
        # Update postvac occupancy - decrement by 1
        clinic.postvac_occupancy_list.append((env.now, clinic.postvac_occupancy_list[-1][1] - 1))
    
    exit_system_ts = env.now    
    print(f"Patient {patient} exited at time {env.now}")

    # Create dictionary of timestamps
    timestamps = {'patient_id': patient,
                  'arrival_ts': arrival_ts,
                  'got_greeter_ts': got_greeter_ts,
                  'release_greeter_ts': release_greeter_ts,
                  'got_reg_ts': got_reg_ts,
                  'release_reg_ts': release_reg_ts,
                  'got_vaccinator_ts': got_vaccinator_ts,
                  'release_vaccinator_ts': release_vaccinator_ts,
                  'got_scheduler_ts': got_scheduler_ts,
                  'release_scheduler_ts': release_scheduler_ts,
                  'exit_system_ts': exit_system_ts}
    
    clinic.timestamps_list.append(timestamps)

Now create a function that runs the clinic for a specified number of hours.

TODO: Dealing with the hours of operation and making sure clinic cleared at end of day. Create clinic level dict of stats such as number of patients vaccinated and end of day timestamp.

In [15]:
def run_clinic(env, clinic, mean_interarrival_time, pct_first_dose, rg, stoptime=simpy.core.Infinity, max_arrivals=simpy.core.Infinity):
      
    # Create a counter to keep track of number of patients generated and to serve as unique patient id
    patient = 0

    # Loop for generating patients
    while env.now < stoptime and patient < max_arrivals:

        # Generate next interarrival time (this will be more complicated later)
        iat = rg.exponential(mean_interarrival_time)
        
        # This process will now yield to a 'timeout' event. This process will resume after iat time units.
        yield env.timeout(iat)

        # New patient generated = update counter of patients
        patient += 1
        
        print(f"Patient {patient} created at time {env.now}")

        env.process(get_vaccinated(env, patient, clinic, pct_first_dose, rg))
        


In [16]:
def main():
    
    # For now we are hard coding the patient arrival rate (patients per hour)
    patients_per_hour = 180
    mean_interarrival_time = 1.0 / (patients_per_hour / 60.0)
    pct_first_dose = 0.50
    
    # Create a random number generator
    rg = default_rng(seed=4470)
    
    # For now we are going to hard code in the resource capacity levels 
    num_greeters = 2
    num_reg_staff = 4
    num_vaccinators = 15
    num_schedulers = 4
    
    # Hours of operation
    stoptime = 600 # No more arrivals after this time
    
    
    # Run the simulation
    env = simpy.Environment()
    # Create a clinic to simulate
    clinic = VaccineClinic(env, num_greeters, num_reg_staff, num_vaccinators, num_schedulers, rg)
    
    env.process(run_clinic(env, clinic, mean_interarrival_time, pct_first_dose, rg, stoptime=stoptime))
    env.run()
    
    # Output log files
    clinic_patient_log_df = pd.DataFrame(clinic.timestamps_list)
    clinic_patient_log_df.to_csv('./output/clinic_patient_log_df.csv', index=False)
    
    vac_occupancy_df = pd.DataFrame(clinic.vac_occupancy_list, columns=['ts', 'occ'])
    vac_occupancy_df.to_csv('./output/vac_occupancy_df.csv', index=False)
    
    postvac_occupancy_df = pd.DataFrame(clinic.postvac_occupancy_list, columns=['ts', 'occ'])
    postvac_occupancy_df.to_csv('./output/postvac_occupancy_df.csv', index=False)
    
    # Note simulation end time
    end_time = env.now
    print(f"Simulation ended at time {end_time}")
    return (end_time)





In [17]:
clinic_end_time = main()


Patient 1 created at time 0.2591639414634509
Patient 2 created at time 0.2883726236719546
Patient 3 created at time 0.4102866904979777
Patient 4 created at time 0.7177108166650722
Patient 5 created at time 0.7737418635168912
Patient 6 created at time 0.8045761087385055
Patient 7 created at time 0.8278647963141194
Patient 8 created at time 0.8730749951751245
Patient 9 created at time 0.9566126281851064
Patient 10 created at time 1.2708129914198512
Patient 11 created at time 1.4715879744085956
Patient 12 created at time 1.9971748612758178
Patient 13 created at time 2.6763806332836673
Patient 14 created at time 2.9490055338099177
Patient 15 created at time 4.344705920264309
Patient 16 created at time 4.941278834221036
Patient 17 created at time 5.350652819656802
Patient 18 created at time 6.2310472804276955
Patient 19 created at time 6.480161462064975
Patient 20 created at time 6.796483482057695
Patient 21 created at time 7.0131050566582545
Patient 22 created at time 7.1735130154022135
Pa

## Redesign for easier use

### Hard coded input parameters

The current version of the model has many hard coded input parameters, including:

* patient arrival rate,
* percent of patients requiring 2nd dose,
* capacity of the various resources (e.g. vaccinators),
* processing time distributions for the various stages of the vaccination process,
* length of time patient needs to remain in observation after being vaccinated (i.e. the 15 minutes),
* length of time to run the simulation model.

This makes it cumbersome to try out different scenarios relating to patient volume and resource capacity.

### Better post-processing

We should make it easy to specify that some post-processing should just happen automatically. In general, we want better control over post-processing and think through how we want to create and store output files.

### Moving code from Jupyter notebook to a Python script

As our code grows, working inside a Jupyter notebook becomes less practical. It's fine for testing out little code snippets, but the production code should be in a `.py` file and we should use an IDE for further development. I've already used the Jupyter notebook export feature to export Model 3 to `vaccine_clinic_model4.py` that we'll use as the basis for further development.

### Adding a command line interface

By moving the code to a `.py` file, it will also be easier to add and use a simple command line interface. This will be useful for automating the process of running numerous scenarios.


## Creating a config file for input parameters (and creating a CLI)

These two enhancements are actually related and we'll address them together. Instead of hard coding input parameter values into our code, we'll store them in a simple configuration file. We'll create a simple example in this notebook and then use the same approach for our production code.

Before getting into configuration files, we need to step back and review passing command line arguments to Python scripts.

### Basic handling of command line arguments (learning about `argv`)

Back in the pcda class, we used some materials from the [Software Carpentry Python Programming lesson](). There were several parts that we left as optional, including the last part on [Command Line Programs](https://swcarpentry.github.io/python-novice-inflammation/12-cmdline/index.html). I highly encourage you to review that first. 

**Bottom line:** `sys.argv` is a list of command line arguments passed to a Python program when running the program.

In [2]:
!python ../src/vaccine_clinic/which_python.py

Python version is 3.9.2 (default, Mar  3 2021, 15:03:14) [MSC v.1916 64 bit (AMD64)]


Let's create a small program that takes a few command line arguments and just repeats them back when the program is run. Here's what the program looks like. I've saved it as `echo_args.py`. We'll interpret the command line arguments as follows:

1. mean interarrival times of patients
2. number of greeters
3. number of registration staff
4. number of vaccinators
5. number of schedulers

We call these *positional arguments* in that our program will infer the meaning of each passed in argument from its position on the command line.

In [ ]:
# echo_args.py
import sys

def main():
    print(f"Command line args: {sys.argv}\n")

    for i, arg in enumerate(sys.argv):
            print(f"sys.argv[{i}]: {arg}")

if __name__ == '__main__':
   main()

If you've forgotten what the purpose of the following is:

```
if __name__ == '__main__':
```

then, make sure you review the [Command Line Programs](https://swcarpentry.github.io/python-novice-inflammation/12-cmdline/index.html) mentioned above.

Also notice the use of `enumerate` with `sys.argv` which allows us to get both the index and argument value from the `sys.argv` list - no need to make our own index loop.

In [11]:
!python ../src/vaccine_clinic/echo_args.py 3.0 2 4 15 4

Command line args: ['../src/vaccine_clinic/echo_args.py', '3.0', '2', '4', '15', '4']

sys.argv[0]: ../src/vaccine_clinic/echo_args.py
sys.argv[1]: 3.0
sys.argv[2]: 2
sys.argv[3]: 4
sys.argv[4]: 15
sys.argv[5]: 4


So, `sys.argv` is a list of the command line arguments passed in to `echo_args.py`. Note that `sys.argv[0]` is just the name of the program itself, including the relative path from this working directory to the program file. At this point, our program doesn't do any checking of the presence or validity of the input arguments. It just prints back out whatever values we input on the command line. If all of these arguments were required, we could add a check like this:

In [ ]:
# echo_args.py
import sys

def main():
    print(f"Command line args: {sys.argv}\n")

    if len(sys.argv) != 6:
        print(f"Five args required, only {len(sys.argv) - 1} specified.")
        exit(1)

    for i, arg in enumerate(sys.argv):
            print(f"sys.argv[{i}]: {arg}")

if __name__ == '__main__':
   main()

In [17]:
!python ../src/vaccine_clinic/echo_args.py 3.0 2 4

Command line args: ['../src/vaccine_clinic/echo_args.py', '3.0', '2', '4']

Five positional args required, only 3 specified.


The example above is highly simplified and the world of command line arguments and command line processing, known as *argument parsing*, is much richer than this. Not only do we have *arguments* such as in this example, we also might have *options* (also called *flags*). In the pcda class we saw this when using things like the `ls` command:

```
ls -l -a
```

We might want to define options for our Python program. These options might appear in any order, if at all. Furthermore,
by convention there are short form flags that start with a single `-` such as `-a` in the example above, and long form options that start with `--` such as `--version`. Often we can use either a short or long form option such as `-h` or `--help`. 

Before checking out argument parsing tools, let's do it ourselves for the following simple case. Let's assume we just want the following command line options. They can be in any order but each must be followed by a numeric value for that option. So,
`argv[i]` will be one of the following for odd values of `i` and `argv[i + 1]` will be the corresponding value.

* `--iat` : mean patient interarrival time
* `--greet` : number of greeters
* `--reg` : number of registration clerks
* `--vacc` : number of vaccinators
* `--sched` : number of schedulers

The following is just a code snippet to illustrate how one might get the input values using standard Python. Once we have the input values, we could pass them on to other parts of our simulation model. Notice that this code doesn't do any input validity checking other than checking for invalid option names. The user input values are stored in a dictionary and this code just prints out that dictionary.

In [ ]:
# get_option_values.py
import sys

def main():

    input_params = {'mean_interarrival_time': 0.0,
                    'num_greeters': 0,
                    'num_reg_staff': 0,
                    'num_vaccinators': 0,
                    'num_schedulers': 0}


    for i, arg in enumerate(sys.argv):
        if arg.startswith('--') and i % 2 > 0:
            if sys.argv[i] == '--iat':
                input_params['mean_interarrival_time'] = sys.argv[i + 1]
            elif sys.argv[i] == '--greet':
                input_params['num_greeters'] = sys.argv[i + 1]
            elif sys.argv[i] == '--reg':
                input_params['num_reg_staff'] = sys.argv[i + 1]
            elif sys.argv[i] == '--vacc':
                input_params['num_vaccinators'] = sys.argv[i + 1]
            elif sys.argv[i] == '--sched':
                input_params['num_schedulers'] = sys.argv[i + 1]
            else:
                print(f"Unrecognized argument: {sys.argv[i]}")

    print(input_params)

if __name__ == '__main__':
   main()

In [14]:
!python ../src/vaccine_clinic/get_option_values.py --iat 3.0 --greet 2 --reg 4 --vacc 15 --sched 4

{'mean_interarrival_time': '3.0', 'num_greeters': '2', 'num_reg_staff': '4', 'num_vaccinators': '15', 'num_schedulers': '4'}


Here we change the order and include one bad option.

In [15]:
!python ../src/vaccine_clinic/get_option_values.py --vacc 15 --iat 3.0 --greet 2 --reg 4 --schedulers 4

Unrecognized argument: --schedulers
{'mean_interarrival_time': '3.0', 'num_greeters': '2', 'num_reg_staff': '4', 'num_vaccinators': '15', 'num_schedulers': 0}


#### Learning more about command line arguments
If you are interested in a deeper dive into command line arguments, check out this tutorial done by the folks at Real Python:

* [Python Command Line Arguments](https://realpython.com/python-command-line-arguments/)

### Tools for argument parsing and building CLI's.

Clearly, it is going to be (potentially) difficult to deal with all the complexity of command line *argument parsing* manually by checking all the values in the `sys.argv` list. Thankfully, there are numerous tools for doing command line argument parsing and helping us create command line interfaces for our Python programs. Some of these tools include:

* [argparse](https://docs.python.org/3/library/argparse.html) - part of the Python standard library
* [click](https://click.palletsprojects.com/en/7.x/) - a popular library for CLIs that uses [function decorators]()
* [fire](https://github.com/google/python-fire) - a newer CLI tool

For our simulation model, we'll use `argparse` to build our CLI since it's built in to Python and is a good thing to learn if you are new to creating CLIs. It's plenty powerful enough for our simple application. Some learning resources for `argparse` include:

* [A "gentle" argparse tutorial](https://docs.python.org/3/howto/argparse.html#id1)
* https://docs.python.org/3/library/argparse.html - the official docs
* https://realpython.com/command-line-interfaces-python-argparse/ - tutorial from Real Python (different that the one on Command Line Arguments mentioned above - this one focuses on argparse)

Before checking out `argparse`, let's do it ourselves for the following simple case. Let's assume we just want the following command line options. They can be in any order but each must be followed by a numeric value for that option. So,
`argv[i]` will be one of the following for odd values of `i` and `argv[i + 1]` will be the corresponding value.

* `--iat` : mean patient interarrival time
* `--greet` : number of greeters
* `--reg` : number of registration clerks
* `--vacc` : number of vaccinators
* `--sched` : number of schedulers

The following is just a code snippet to illustrate how one might get the input values using standard Python. Once we have the input values, we could pass them on to other parts of our simulation model. Notice that this code doesn't do any input validity checking other than checking for invalid option names. The user input values are stored in a dictionary and this code just prints out that dictionary.

In [18]:
# get_option_values.py
import sys

def main():

    input_params = {'mean_interarrival_time': 0.0,
                    'num_greeters': 0,
                    'num_reg_staff': 0,
                    'num_vaccinators': 0,
                    'num_schedulers': 0}


    for i, arg in enumerate(sys.argv):
        if arg.startswith('--') and i % 2 > 0:
            if sys.argv[i] == '--iat':
                input_params['mean_interarrival_time'] = sys.argv[i + 1]
            elif sys.argv[i] == '--greet':
                input_params['num_greeters'] = sys.argv[i + 1]
            elif sys.argv[i] == '--reg':
                input_params['num_reg_staff'] = sys.argv[i + 1]
            elif sys.argv[i] == '--vacc':
                input_params['num_vaccinators'] = sys.argv[i + 1]
            elif sys.argv[i] == '--sched':
                input_params['num_schedulers'] = sys.argv[i + 1]
            else:
                print(f"Unrecognized argument: {sys.argv[i]}")

    print(input_params)

if __name__ == '__main__':
   main()

{'mean_interarrival_time': 0.0, 'num_greeters': 0, 'num_reg_staff': 0, 'num_vaccinators': 0, 'num_schedulers': 0}


In [19]:
main()

{'mean_interarrival_time': 0.0, 'num_greeters': 0, 'num_reg_staff': 0, 'num_vaccinators': 0, 'num_schedulers': 0}


In [14]:
!python ../src/vaccine_clinic/get_option_values.py --iat 3.0 --greet 2 --reg 4 --vacc 15 --sched 4

{'mean_interarrival_time': '3.0', 'num_greeters': '2', 'num_reg_staff': '4', 'num_vaccinators': '15', 'num_schedulers': '4'}


Here we change the order and include one bad option.

In [15]:
!python ../src/vaccine_clinic/get_option_values.py --vacc 15 --iat 3.0 --greet 2 --reg 4 --schedulers 4

Unrecognized argument: --schedulers
{'mean_interarrival_time': '3.0', 'num_greeters': '2', 'num_reg_staff': '4', 'num_vaccinators': '15', 'num_schedulers': 0}


### Creating a CLI with argparse

https://realpython.com/command-line-interfaces-python-argparse/

https://realpython.com/python-main-function/

In [ ]:
def process_command_line(argv=None):
    """
    Parse command line arguments
    
    `argv` is a list of arguments, or `None` for ``sys.argv[1:]``.
    Return a Namespace representing the argument list.
    """

    # If argv is empty, get the argument list from sys.argv.
    if argv is None:
        argv = sys.argv[1:]

    # Create the parser
    parser = argparse.ArgumentParser(prog='vaccine_clinic_model4',
                                     description='Run vaccine clinic simulation')

    # Add arguments
    parser.add_argument("patient_arrival_rate", help="patients per hour",
                        type=float)

    parser.add_argument("num_greeters", help="number of greeters",
                        type=int)

    parser.add_argument("num_reg_staff", help="number of registration staff",
                        type=int)

    parser.add_argument("num_vaccinators", help="number of vaccinators",
                        type=int)

    parser.add_argument("num_schedulers", help="number of schedulers",
                        type=int)

    parser.add_argument(
        "--scenario", type=str, default=datetime.now().strftime("%Y.%m.%d.%H.%M."),
        help="Prepended to output filenames."
    )

    parser.add_argument("--pct_need_second_dose", default=0.5,
                        help="percent of patients needing 2nd dose (default = 0.5)",
                        type=float)

    parser.add_argument("--stoptime", default=600, help="time that simulation stops (default = 600)",
                        type=float)

    parser.add_argument(
        "--output-path", type=str, default="", help="location for output file writing")

    parser.add_argument("--quiet", action='store_true',
                        help="If True, suppresses output messages (default=False")

    # do the parsing
    args = parser.parse_args()
    return args